**Required packages**

In [ ]:
!pip install -I pillow==7.2.0
!sudo apt-get update
!pip install pdf2image
!apt-get install poppler-utils 
!sudo apt install tesseract-ocr
!sudo apt-get install tesseract-ocr-all
!pip install cv2
!pip install pytesseract
!pip install arabic-reshaper
!pip install PyArabic
!pip install pypdfium2
from pdf2image import convert_from_path
import pytesseract
import shutil
import os
import os.path
from os import path
import random
from PIL import Image
import cv2 
import arabic_reshaper
from pyarabic.araby import strip_tatweel, strip_tashkeel, normalize_hamza
import re
import pypdfium2 as pdfium
from itertools import chain

**Functions**

In [45]:
# get text from pdf document
def get_text(f_path, language, ocr):

    text_all = '' 

    if ocr == True:
        # convert pdf -> images
        images = convert_from_path(f_path) 

        # loop over images
        i = 1
        for img in images:

            # limit to first and last 20 pages & ignore first page
            if (i >= 2 & i <= 20) | (i >= len(images) - 20  & i <= len(images)):

                if os.path.exists(re.sub('.pdf', '/', f_path)) == False: # create directory
                    os.mkdir(re.sub('.pdf', '/', f_path))
                img_path = re.sub('.pdf', '/', f_path) + str(i) + '.jpg'
                img.save(img_path, 'JPEG') # save image
                    
                text = pytesseract.image_to_string(cv2.imread(img_path), lang = language) # ocr

                text_all = text_all + '\n' + text # store text
            i = i + 1

    else:
          # open pdf document
          pdf = pdfium.PdfDocument(f_path) 

          # limit to first and last 20 pages & ignore first page
          if len(pdf) <= 20:
              n_pages = range(1, len(pdf))
          else:
              n_pages = chain(range(1, 20), range(len(pdf)-20, len(pdf)))

          # loop over pages
          for i in n_pages:

              page = pdf[i] # get page

              textpage = page.get_textpage()
              text = textpage.get_text_range() # extract text

              text = re.sub('\r\n', ' ', text) # clean text

              text_all = text_all + '\n' + text # store text

    return text_all

# clean text from noise
def clean(text):

    new_text = ''

    # define patterns
    regex = 'dédicas|dedicas|notation et symboles|remerciements|remerciement|acknowledgement|dédicaces|dedicaces|dédicace|dedicace|liste des figures|liste-des-figures|list of figures|list-of-figures|liste des abreviations|liste-des-abreviations|liste des abréviations|liste-des-abréviations|list of abbreviations|list-of-abbreviations|table des matières|table des matieres|table-des-matières|table-des-matieres|sommaire|table of contents|table-of-contents|liste des tableaux|liste-des-tableaux|list of tables|list-of-tables|annexe|références|references|bibliographie|références bibliographiques|références-bibliographiques|bibliographic references|bibliographic-references'
    ar_regex = arabic_reshaper.reshape('المراجع|اهدﺍﺀ|ﺷﻜﺮ ﻭﺗﻘﺪﯾﺮ|قائمة-الاشكال|قائمة الاشكال|تقدير|قائمة-الاختصارات|قائمة الاختصارات|قائمة جدول|قائمة جداول|قائمة-جداول|قائمة-جدول|فهرس|الفهرس|جدول المحتويات|قائمة المحتويات|قائمة-المحتويات|جدول-المحتويات|ﺷﻜﺮ ﻭ ﻋﺮﻓﺎﻥ|ﺷﻜﺮ ﻭﻋﺮﻓﺎﻥ|ﺷﻜﺮ ﻭ ﺗﻘﺪﯾﺮ|فهرس ﺍﻟﻤﻮﺍﺿﻴﻊ')
     
    # loop over pages
    text = text.split('\n')
    for p in text: 

        if re.search(regex, p, re.IGNORECASE) or re.search(ar_regex, p, re.IGNORECASE): # remove outlier pages
            pass

        else:

            p = arabic_reshaper.reshape(p) # reshape arabic
            # remove diactrics
            p = strip_tatweel(p)
            p = strip_tashkeel(p)

            new_text = new_text + '\n' + p # store text
   
    return new_text

In [55]:
# find abstract by regex
def get_abstract(text):
    
    en_abstract = ''
    fr_abstract = ''
    ar_abstract = ''

    # define patterns
    en_regex = 'abstract|summary'
    fr_regex = 'résumé|resume|résume|resumé'
    ar_regex = arabic_reshaper.reshape('ملخص|الملخص')

    # loop over pages
    pages = text.split('\n')
    for p in pages:

        last_pos = len(p) # get last index
        mtch = re.search(en_regex, p, re.IGNORECASE)
        if mtch and en_abstract == '': # extract english abstract
            position = mtch.start()
            en_abstract = p[position : min(last_pos, position + 3000)] 
            if len(en_abstract) <= 50: en_abstract = '' 

        mtch = re.search(fr_regex, p, re.IGNORECASE)
        if mtch and fr_abstract == '': # extract french abstract
            position = mtch.start()
            fr_abstract = p[position : min(last_pos, position + 3000)] 
            if len(fr_abstract) <= 50: fr_abstract = '' 
            
        mtch = re.search(ar_regex, p, re.IGNORECASE)
        if mtch and ar_abstract == '': # extract arabic abstract
            position = mtch.start()
            ar_abstract = p[position : min(last_pos, position + 3000)]     
            if len(ar_abstract) <= 50: ar_abstract = '' 

    return [en_abstract, fr_abstract, ar_abstract]

# find keywords by regex
def get_keywords(text):

    en_keywords = ''
    fr_keywords = ''
    ar_keywords = ''
    
    # define patterns
    en_regex = 'keywords|key words|key-words|index terms|index-terms'
    fr_regex = 'mots clés|mots clé|mots-clés|mots-clé|mots cles|mots cle|mots-cles|mots-cle|termes indexation|termes-indexation'
    ar_regex = arabic_reshaper.reshape('الكلمات المفتاحية|الكلمات-المفتاحية|الكلمات الافتتاحية|الكلمات-الافتتاحية|الكلمات المفتاح|الكلمات-المفتاح|كلمات مفتاحية|كلمات-مفتاحية|كلمات افتتاحية|كلمات-افتتاحية|كلمات مفتاح|كلمات-مفتاح')

    # loop over pages
    pages = text.split('\n')
    for p in pages:

        last_pos = len(p) # get last index
        mtch = re.search(en_regex, p, re.IGNORECASE)
        if mtch and en_keywords == '': # extract english keywords
            position = mtch.start()
            en_keywords = p[position : min(last_pos, position + 250)] 

        mtch = re.search(fr_regex, p, re.IGNORECASE)
        if mtch and fr_keywords == '': # extract french keywords
            position = mtch.start()
            fr_keywords = p[position : min(last_pos, position + 250)] 
            
        mtch = re.search(ar_regex, p, re.IGNORECASE)
        if mtch and ar_keywords == '': # extract arabic keywords
            position = mtch.start()
            ar_keywords = p[position : min(last_pos, position + 250)]  

    return [en_keywords, fr_keywords, ar_keywords]

****

**Main**

In [62]:
def main(f_path, lang = 'fra', ocr = False): 
  
    text = clean(get_text(f_path, lang, ocr))  # by default extract without ocr
    ena, fra, ara = get_abstract(text)
    enk, frk, ark = get_keywords(text)

    regex = 'keywords|key words|key-words|index terms|index-terms|mots cles|mots cle|mots-cles|mots-cle|mots clés|mots clé|mots-clés|mots-clé|termes indexation|termes-indexation'
    ar_regex = arabic_reshaper.reshape('الكلمات المفتاحية|الكلمات-المفتاحية|الكلمات الافتتاحية|الكلمات-الافتتاحية|الكلمات المفتاح|الكلمات-المفتاح|كلمات مفتاحية|كلمات-مفتاحية|كلمات افتتاحية|كلمات-افتتاحية|كلمات مفتاح|كلمات-مفتاح')
    if re.search(regex, ena, re.IGNORECASE): ena = ena[0: re.search(regex, ena, re.IGNORECASE).start()]
    if re.search(ar_regex, ena, re.IGNORECASE): ena = ena[0: re.search(ar_regex, ena, re.IGNORECASE).start()]
    if re.search(regex, fra, re.IGNORECASE): fra = fra[0: re.search(regex, fra, re.IGNORECASE).start()]
    if re.search(ar_regex, fra, re.IGNORECASE): fra = fra[0: re.search(ar_regex, fra, re.IGNORECASE).start()]
    if re.search(regex, ara, re.IGNORECASE): ara = ara[0: re.search(regex, ara, re.IGNORECASE).start()]
    if re.search(ar_regex, ara, re.IGNORECASE): ara = ara[0: re.search(ar_regex, ara, re.IGNORECASE).start()]
    if re.search(regex, enk, re.IGNORECASE): enk = enk[re.search(regex, enk, re.IGNORECASE).end() ::]
    if re.search(regex, frk, re.IGNORECASE): frk = frk[re.search(regex, frk, re.IGNORECASE).end() ::]
    if re.search(ar_regex, ark, re.IGNORECASE): ark = ark[re.search(ar_regex, ark, re.IGNORECASE).end() ::]

    regex = 'abstract|summary|résumé|resumé|résume|resume'
    ar_regex = arabic_reshaper.reshape('ملخص|الملخص')
    if re.search(regex, enk, re.IGNORECASE): enk = enk[0: re.search(regex, enk, re.IGNORECASE).start()]
    if re.search(ar_regex, enk, re.IGNORECASE): enk = enk[0: re.search(ar_regex, enk, re.IGNORECASE).start()]
    if re.search(regex, frk, re.IGNORECASE): frk = frk[0: re.search(regex, frk, re.IGNORECASE).start()]
    if re.search(ar_regex, frk, re.IGNORECASE): frk = frk[0: re.search(ar_regex, frk, re.IGNORECASE).start()]
    if re.search(regex, ark, re.IGNORECASE): ark = ark[0: re.search(regex, ark, re.IGNORECASE).start()]
    if re.search(ar_regex, ark, re.IGNORECASE): ark = ark[0: re.search(ar_regex, ark, re.IGNORECASE).start()]
    if re.search(regex, ena, re.IGNORECASE): ena = ena[re.search(regex, ena, re.IGNORECASE).end() ::]
    if re.search(regex, fra, re.IGNORECASE): fra = fra[re.search(regex, fra, re.IGNORECASE).end() ::]
    if re.search(ar_regex, ara, re.IGNORECASE): ara = ara[re.search(ar_regex, ara, re.IGNORECASE).end() ::]
    
    ena = ena.lstrip()
    ena = ena.lstrip(':')
    ena = ena.lstrip()
    fra = fra.lstrip()
    fra = fra.lstrip(':')
    fra = fra.lstrip()
    ara = ara.lstrip()
    ara = ara.lstrip(':')
    ara = ara.lstrip()
    enk = enk.lstrip()
    enk = enk.lstrip(':')
    enk = enk.lstrip()
    frk = frk.lstrip()
    frk = frk.lstrip(':')
    frk = frk.lstrip()
    ark = ark.lstrip()
    ark = ark.lstrip(':')
    ark = ark.lstrip()

    abstract = ena + '\n' + fra + '\n' + ara
    keywords = enk + '\n' + frk + '\n' + ark

    return [abstract, keywords, text]